In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [16]:
df_citylab_rdy = pd.read_csv("Citylab-Berlin_Projekte.csv", sep=";")
df_citylab_rdy.head()

,Quelle,Projektname,Art,Einsatzbereich,Webseite-Link,Organisation,Ansprechperson,Email,Status,Kurzzusammenfassung,Unnamed: 10


***
## Main Page

In [18]:
url = 'https://citylab-berlin.org/de/projects/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
first_chunks = soup.find_all('div', class_='wpgb-masonry')

laufend_chunk = first_chunks[0].find_all('article')
abgeschlossen_chunk = first_chunks[1].find_all('article')

In [20]:
laufend_dict = {}

for tile in laufend_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text
    
    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)
    
    laufend_dict[title] = {
        "sub-link" : sub_domain_link,
        "Einsatzbereich" : categeory_texts
    } 

for key in list(laufend_dict.keys()):
    split_string = re.split(r'[\\]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    laufend_dict[new_key] = laufend_dict.pop(key)

laufend_dict

{'BärGPT': {'sub-link': 'https://citylab-berlin.org/de/projects/baergpt/',
  'Einsatzbereich': ['Digitale Zusammenarbeit', 'Neue Technologien']},
 'Data Hub Berlin': {'sub-link': 'https://citylab-berlin.org/de/projects/data-hub-berlin/',
  'Einsatzbereich': ['Innovative Verwaltung', 'Offene Daten', 'Open Source']},
 'Partizipation Digital': {'sub-link': 'https://citylab-berlin.org/de/projects/partizipation-digital/',
  'Einsatzbereich': ['Digitale Zusammenarbeit', 'Innovative Verwaltung']},
 'GovTech TestLAB': {'sub-link': 'https://citylab-berlin.org/de/projects/govtech-testlab/',
  'Einsatzbereich': ['Digitale Zusammenarbeit',
   'Innovative Verwaltung',
   'Open Source']},
 'Stadtlabor2Go': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtlabor2go/',
  'Einsatzbereich': ['Digitale Zusammenarbeit',
   'Innovative Verwaltung',
   'Offene Daten, offene Städte']},
 'Fairgnügen': {'sub-link': 'https://citylab-berlin.org/de/projects/fairgnuegen/',
  'Einsatzbereich': ['Digitale Zu

In [21]:
abgeschlossen_dict = {}

for tile in abgeschlossen_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text

    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)

    abgeschlossen_dict[title] = {
        "sub-link" : sub_domain_link,
        "Einsatzbereich" : categeory_texts
    } 

for key in list(abgeschlossen_dict.keys()):
    split_string = re.split(r'[\–\-\:]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    abgeschlossen_dict[new_key] = abgeschlossen_dict.pop(key)

abgeschlossen_dict

{'Digital Vereint': {'sub-link': 'https://citylab-berlin.org/de/projects/digitalvereint/',
  'Einsatzbereich': ['Smart Cities für alle']},
 'Quantified Trees (Qtrees)': {'sub-link': 'https://citylab-berlin.org/de/projects/qtrees/',
  'Einsatzbereich': ['Energie und Nachhaltigkeit']},
 'Verwaltungsdashboard': {'sub-link': 'https://citylab-berlin.org/de/projects/afs-dashboard/',
  'Einsatzbereich': ['Innovative Verwaltung']},
 'Stadtpuls': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtpuls/',
  'Einsatzbereich': ['Offene Daten, offene Städte', 'Smart Cities für alle']},
 'Pixelwelt': {'sub-link': 'https://citylab-berlin.org/de/projects/pixelwelt/',
  'Einsatzbereich': ['Digitale Zusammenarbeit']},
 'Prototypenwerkstatt': {'sub-link': 'https://citylab-berlin.org/de/projects/prototypenwerkstatt/',
  'Einsatzbereich': ['Innovative Verwaltung']},
 'Prozessanalyse Radinfrastruktur (PARI)': {'sub-link': 'https://citylab-berlin.org/de/projects/prozessanalyse-radinfrastruktur-pari/',

*** 
## Subdomains

In [22]:
import json

abgeschlossen_soup_dict = {}

for key, values in abgeschlossen_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    abgeschlossen_soup_dict[key] = str(subsoup)

In [23]:
with open('abgeschlossen_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(abgeschlossen_soup_dict, f)

In [24]:
laufend_soup_dict = {}

for key, values in laufend_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    laufend_soup_dict[key] = str(subsoup)

In [25]:
with open('laufend_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(laufend_soup_dict, f)

In [118]:
for key, values in abgeschlossen_dict.items():
    subsoup = BeautifulSoup(abgeschlossen_soup_dict[key], 'html.parser')

    footer_div = subsoup.find('div', class_="site-footer")
    if footer_div:
        footer_div.extract()
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if web_link is None:
        web_link = abgeschlossen_dict[key]['sub-link']
    
    kontakt_header = subsoup.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'], string=lambda text: text and 'kontakt' in text.strip().lower())
    kontakt_container = kontakt_header.find_next('p')

    contacts = kontakt_container.find_all('a')
    emails = []
    kontakte = []
    for contact in contacts:
        contact_text = contact.get_text(strip=True)
        kontakte.append(contact_text)

        href = contact.get('href')
        # Special case for "CityLAB Berlin" - hardcoded like a corpo backdoor
        if contact_text == "CityLAB Berlin":
            emails.append("info@citylab-berlin.org")
        # Extract email from mailto: links
        elif href.startswith('mailto:'):
            email = href.replace('mailto:', '').strip()
            emails.append(email)
        # Capture any other hrefs too, bredrin
        elif href:
            emails.append(href)

    status = "Projekt Abgeschlossen"

    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Webseite-Link": web_link,
        "Ansprechperson": kontakte,
        "Email": emails,
        "Status": status,
        "Kurzzusammenfassung": text
    })

In [119]:
for key, values in laufend_dict.items():
    subsoup = BeautifulSoup(laufend_soup_dict[key], 'html.parser')

    footer_div = subsoup.find('div', class_="site-footer")
    if footer_div:
        footer_div.extract()
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if web_link is None:
        web_link = laufend_dict[key]['sub-link']
    
    kontakt_header = subsoup.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'], string=lambda text: text and 'kontakt' in text.strip().lower())
    if kontakt_header:
        kontakt_container = kontakt_header.find_next('p')

    contacts = kontakt_container.find_all('a')
    emails = []
    kontakte = []
    for contact in contacts:
        contact_text = contact.get_text(strip=True)
        kontakte.append(contact_text)

        href = contact.get('href')
        # Special case for "CityLAB Berlin" - hardcoded like a corpo backdoor
        if contact_text == "CityLAB Berlin":
            emails.append("info@citylab-berlin.org")
        # Extract email from mailto: links
        elif href.startswith('mailto:'):
            email = href.replace('mailto:', '').strip()
            emails.append(email)
        # Capture any other hrefs too, bredrin
        elif href:
            emails.append(href)

    status = "Laufend"

    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Webseite-Link": web_link,
        "Ansprechperson": kontakte,
        "Email": emails,
        "Status": status,
        "Kurzzusammenfassung": text
    })

In [120]:
with open('laufend_dict_working.json', 'w', encoding='utf-8') as f:
    json.dump(laufend_dict, f, ensure_ascii=False, indent=4)

with open('abgeschlossen_dict_working.json', 'w', encoding='utf-8') as f:
    json.dump(abgeschlossen_dict, f, ensure_ascii=False, indent=4)

In [128]:
df_abgeschlossen = pd.DataFrame()

df_abgeschlossen["Projektname"] = abgeschlossen_dict.keys()
values_df = pd.DataFrame(list(abgeschlossen_dict.values()))
df_abgeschlossen = pd.concat([df_abgeschlossen, values_df], axis=1)

In [129]:
df_laufend = pd.DataFrame()

df_laufend["Projektname"] = laufend_dict.keys()
values_df = pd.DataFrame(list(laufend_dict.values()))
df_laufend = pd.concat([df_laufend, values_df], axis=1)

In [161]:
df_citylab_working = pd.DataFrame()

df_citylab_working = pd.concat([df_laufend, df_abgeschlossen])
df_citylab_working.reset_index(drop=True, inplace=True)
df_citylab_working.drop('sub-link', axis=1, inplace=True)

In [162]:
df_citylab_working["Quelle"] = "CityLAB-Berlin: https://citylab-berlin.org"
df_citylab_working["Organisation"] = "CityLAB-Berlin"
df_citylab_working["Art"] = None

In [163]:
df_citylab_working.to_csv('Citylab-Berlin_Projekte.csv', index=False)